In [5]:
import psycopg2
import configparser
from sql_queries import Data_quality_queries

In [7]:
config = configparser.ConfigParser()
config.read("parameters.cfg")

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['postgresql_sql'].values()))
cur = conn.cursor()

In [26]:
for query in Data_quality_queries:
    cur.execute(query)
    records = cur.fetchone()
    table = query.replace("SELECT COUNT (*) FROM ","")
    table = table.replace(";","")
    if records[0] < 1:
        print("1")
        raise ValueError(f"Data quality check failed. query {table} returned 0 rows")
    else:
        print(f"Data quality for table \"{table}\" check passed with {records[0]} records")

Data quality for table "dim_countries" check passed with 227 records
Data quality for table "dim_exposure" check passed with 191 records
Data quality for table "dim_vaccination" check passed with 54341 records
Data quality for table "fact_covid" check passed with 2214879 records
